### Code to retrieve parameters from a calculation and estimate final energy and error

In [1]:
%load_ext autoreload
%autoreload 2
#import plum
#plum.autoreload.activate()

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import time
import json

from functools import partial, reduce

In [3]:
# Added to silence some warnings.
from jax.config import config
config.update("jax_enable_x64", True)

import jax
import jax.numpy as jnp

import flax
import optax

/opt/anaconda3/envs/q_embedding_new/lib/python3.8/site-packages/flax/core/frozen_dict.py:169: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(


In [4]:
import pennylane as qml
import netket as nk
from netket.operator.spin import sigmax,sigmaz

In [5]:
np.set_printoptions(precision=4)
np.set_printoptions(linewidth=250)

In [6]:
from utils                  import *
from hamiltonians           import *
from quantum_circuits       import *
from classic_models         import *
from truncated_spin         import Spin

## Ammonia - System and Hamiltonian

In [7]:
phys_qubits       = 4
n_classical_spins = 10

ancilla_qubits    = 1
tot_qubits        = phys_qubits + ancilla_qubits

q_index           = [0,1,2,3]
q_str             = "".join([str(orb) for orb in q_index])
c_index           = [4,5,6,7,8,9,10,11,12,13]
tot_spins         = n_classical_spins + phys_qubits
tot_elec_classic  = 6 #number of electrons in the classical part
tot_elec_quantum  = 2 #number of electrons in the quantum part (always 2 in the HONO/LUNO active space)
tot_elec          = tot_elec_classic+tot_elec_quantum

In [8]:
device_phys         = qml.device("default.qubit", wires=list(range(phys_qubits)))
device_with_ancilla = qml.device("default.qubit", wires=list(range(phys_qubits))+["a"])

# Define the Hilbert space by specifying the maximum and minimum amount of electrons with spin up and down

max_up = tot_elec//2
if max_up >  n_classical_spins//2:
    max_up = n_classical_spins//2

min_up = tot_elec//2-phys_qubits//2

print("Max up:",max_up,"min up:", min_up)
# Particle preserving hilbert space
valid_samples = True # If True, the samples are generated with the correct number of particles
hi = Spin(N=n_classical_spins//2, maximum_up=max_up,minimum_up=min_up) * Spin(N=n_classical_spins//2, maximum_up=max_up,minimum_up=min_up)

Max up: 4 min up: 2


In [9]:
## Prepare the Hamiltonian

m_name     = "NH3"
orbit_list = [0]
orbit_str  = "".join([str(orb) for orb in orbit_list])
distance   = 1.5

mol_file = "./../../data/ammonia/NH3_qubit_op/NH3_"+orbit_str+"reduced/"+m_name+"_qubit_op_"+str(tot_spins)+"qubits_"+str(distance)+"r_"+orbit_str+"reduced"+"_aug-cc-pvqz_iao.dat"

In [10]:
h_mixed = mixed_hami_from_file(mol_file, hi, tot_spins, q_index, c_index)

Number of pure  quantum op: 27
Number of pure  classic op: 443
Number of       mixed   op: 1272
Number of quantum    group: 76


## Retrieve the ansatz

In [15]:
alpha_a  = 2
angles_a = 2
alpha_c  = 1

tot_steps = 1500
n_samples = 20000

In [16]:
model_file = "./../../data/ammonia/NH3_hybrid_parameters/q_embedding_"+str(phys_qubits)+"qubits_"+str(n_classical_spins)+"spins_"+str(alpha_c)+"alpha_"+str(angles_a)+"angles_"+orbit_str+"orbitals_"+q_str+"active_"+str(tot_steps)+"steps_"+str(n_samples)+"samples.dat"

In [17]:
with open(model_file,"rb") as f:
    model_data = pickle.load(f)

In [34]:
model_data.keys()

dict_keys(['c_index', 'q_index', 'mixed_energies', 'mixed_gradients', 'reference_energy', 'relative_error', 'pars_best', 'pars_final', 'sigma_best', 'seed', 'sampler_state', 'optimizer', 'optimizer_state', 'quantum_model', 'classic_model', 'angles_model'])

In [31]:
model_data["pars_best"]

FrozenDict({
    angles: {
        params: {
            Dense: {
                bias: Array([ 0.6741, -0.3537,  0.5348, -0.6373, -0.5109, -0.0281,  0.9835, -0.4785,  0.2269, -0.2992,  0.5896,  1.0395, -0.2289,  0.1047, -0.3806,  0.9168,  0.4365, -0.4076, -1.003 ,  1.1048], dtype=float64),
                kernel: Array([[-0.8194, -0.8804, -0.0905, -0.2375,  0.4889,  0.9277, -0.1962,  0.975 , -1.0195,  1.1178, -0.6507, -0.8347, -0.3416,  0.4028,  1.1731, -0.0639,  1.3285, -0.2094,  1.3769, -0.1502],
                       [ 0.4653,  0.5987,  0.3296, -0.0829,  0.069 , -0.8186, -0.4015, -0.506 ,  0.3185,  1.1427,  0.2737, -0.1486, -0.1054, -0.2727, -0.262 , -0.4277,  0.2857,  0.5694,  0.4072,  0.202 ],
                       [ 0.907 ,  0.6003, -0.3723, -0.5692,  0.3661, -0.1072,  0.9853,  0.0163, -0.0648, -0.1702,  0.9575,  0.248 ,  0.7744, -0.2922,  0.2894,  1.0408,  0.3323,  1.5168, -0.8626,  1.6712],
                       [ 0.1081, -0.2005, -0.7067, -0.4388, -0.1941,  0.0968, -0.1623

## Final energy

In [32]:
print(model_data["mixed_energies"][-1])

-56.2080+0.0000j ± 0.0012 [σ²=0.0274]
